In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [2]:
import sys

linewidth = 100
formatter = {
    'float': lambda num: f'{num:5.1f}',
    'int': lambda num: f'{float(num):5.1f}',
}
print_args = {
    'threshold': sys.maxsize,
#     'threshold': 100,
    'linewidth': 100,
    'formatter': formatter,
    'edgeitems': 6,
}

In [3]:
import numpy as np
from qtm.constants import RYDBERG

from qtm.lattice import RealLattice, ReciLattice
from qtm.gspace import GSpace, GkSpace

# Iron Lattice
alat = 5.107
latvec_alat = 0.5 * np.array([
    [ 1,  1,  1],
    [-1,  1,  1],
    [-1, -1,  1]
])

reallat = RealLattice.from_alat(alat, *latvec_alat)
recilat = ReciLattice.from_reallat(reallat)


# Creating the Serial GSpace instances
ecutwfn = 40 * RYDBERG
grho = GSpace(recilat, 4 * ecutwfn)
gkwfn = GkSpace(grho, [0., 0., 0.])

In [4]:
# Creating test data for FFT operations
gspc = grho

arr_g = gspc.create_buffer_g(1)
arr_r = gspc.create_buffer_r(1)

# Random Field is constructed in G-Space
rng = np.random.default_rng(12345)
arr_g[:] = rng.random(gspc.size_g) \
    + 1j * rng.random(gspc.size_g)

# Conversion to real-space grid done with NumPy
arr_r[:] = 0
arr_r.put(gspc.idxgrid, arr_g)
arr_r = np.fft.ifftn(arr_r.reshape(gspc.grid_shape))
arr_r = arr_r.reshape((1, -1))

# Running the gspc's FFT operations and comparing
# with generated field pair
out = gspc.create_buffer_r(1)
gspc.g2r(arr_g, out)
print(np.allclose(out, arr_r))

out = gspc.create_buffer_g(1)
gspc.r2g(arr_r, out)
print(np.allclose(out, arr_g))

True
True


In [6]:
from qtm.mpi.comm import QTMComm
from qtm.mpi.gspace import DistGSpace
from qtm.gspace.fft.utils import cryst2idxgrid

# Creating a dummy comm to test
# how the G-vectors are distributed
# Actually testing FFT operations requires mpi4py
class DummyComm(QTMComm):
    
    def __init__(self, size, rank):
        self.size, self.rank = size, rank
    
    def Get_size(self):
        return self.size
    
    def Get_rank(self):
        return self.rank

# Generating DistGSpaceBase instances for different ranks
# and collecting its list of gspc G-vectors it owns.
size = 6

nx, ny, nz = gspc.grid_shape
idxgrid = gspc.idxgrid
ix, iy, iz = np.unravel_index(idxgrid, gspc.grid_shape, order='C')

l_iglocal = []
for rank in range(size):
    comm = DummyComm(size, rank)
    dist_gspc = DistGSpace(comm, gspc)
    # print(dist_gspc.size_g)
    l_iglocal.append(dist_gspc.ig_loc)
    # print(dist_gspc._glob2dist_counts)
print(l_iglocal)
# igall = np.concatenate(l_iglocal)
# print(np.all(igall[dist_gspc._gather2glob] 
#              == np.arange(gspc.size_g, dtype='i8')))

True
True
True
True
True
True
[slice(0, 466, None), slice(466, 855, None), slice(855, 1178, None), slice(1178, 1435, None), slice(1435, 1804, None), slice(1804, 2243, None)]


In [ ]:
a = np.arange(24).reshape((2,3,4))
print(a.transpose((1,2,0)).shape)

In [9]:
from qtm.mpi.gspace import DistGkSpace

l_iglocal = []
for rank in range(size):
    comm = DummyComm(size, rank)
    dist_gspc = DistGkSpace(comm, gspc)
    # print(dist_gspc.size_g)
    l_iglocal.append(dist_gspc.ig_loc)
    
igall = np.concatenate(l_iglocal)
print(np.all(igall[dist_gspc._gather2glob] 
             == np.arange(gspc.size_g, dtype='i8')))

TypeError: 'gkspc' must be a '<class 'qtm.gspace.gkspc.GkSpace'>' instance. got '<class 'qtm.gspace.gspc.GSpace'>'.

In [ ]:
import ipyparallel as ipp
print(ipp.version_info)

nproc = 4
cluster = ipp.Cluster(engines='mpi', n=nproc, shutdown_atexit=False)
print(cluster)
client = cluster.start_and_connect_sync(activate=True)
view = client[:]
client.ids

In [ ]:
!ipcluster list

In [ ]:
%autopx --block --group-outputs=engines

In [6]:
import numpy as np
from qtm.mpi.utils import scatter_range

nsplit = 3
a = np.array([0,0,0,0,0,1,1,1,1,2,2,2,2,3,3,4,4,5,5,6,6,6,6,7])
idx = np.unique(a)

for isplit in range(nsplit):
    iloc = scatter_range(len(idx), nsplit, isplit)
    # print(idx[iloc])
    istart = np.searchsorted(a, idx[iloc.start])
    istop = np.searchsorted(a, idx[iloc.stop-1], 'right')
    print(idx[iloc], a[istart:istop])

[0 1 2] [0 0 0 0 0 1 1 1 1 2 2 2 2]
[3 4 5] [3 3 4 4 5 5]
[6 7] [6 6 6 6 7]
